## Install packages

In [1]:
# !pip install xmltodict
# !pip install untangle
# !pip install pandas

## Import libraries

In [204]:
import json, xmltodict, os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Functions

In [160]:
path_en = 'pan19-author-profiling-earlybirds-20190320/en/'
path_es = 'pan19-author-profiling-earlybirds-20190320/es/'
path_to_save_en = 'JSON/pan19-author-profiling-earlybirds-20190320/en/'
path_to_save_es = 'JSON/pan19-author-profiling-earlybirds-20190320/es/'
path_labels_en = 'pan19-author-profiling-earlybirds-20190320/en.txt'
path_labels_es = 'pan19-author-profiling-earlybirds-20190320/es.txt'

In [36]:
def get_list_names(path):

    file_names = []

    for filename in os.listdir(path):
        f = os.path.join(path, filename)

        if os.path.isfile(f):
            if '.txt' not in filename:
                file_names.append(filename.split('.')[0])
    
    return file_names

In [37]:
file_names_en = get_list_names(path_en)
file_names_es = get_list_names(path_es)

In [55]:
# Function to check file names 
def check_path(file_name):
    if file_name[0].isdigit():
        # return os.path.join(f'\', file_name)
        return file_name
    else:
        return file_name

In [14]:
def xml_to_json(path, file_names, path_to_save):

    for i, file_name in enumerate(file_names):
        file_name = check_path(file_name)
        final_path = f'{path}{file_name}.xml'

        # Reading XML file and converting it into text
        with open(final_path, encoding = 'utf-8') as xml_file:
            data = xmltodict.parse(xml_file.read())
        
        # Converting text into JSON style
        data = json.dumps(data, indent = 2)

        # Saving JSON file
        saving_path = f'{path_to_save}{file_name}.json'
        with open(saving_path, 'w') as json_file:
            json_file.write(data)

In [15]:
xml_to_json(path_en, file_names_en, path_to_save_en)
xml_to_json(path_es, file_names_es, path_to_save_es)

In [196]:
def get_labels(labels_path, file_name):
    # Leyendo y creando un DF de las etiquetas del archivo JSON
    df_labels = pd.read_csv(labels_path, sep = ':::', names = ['id', 'author', 'gender'], 
                        engine = 'python')
    
    author = df_labels[df_labels.id == file_name].author
    gender = df_labels[df_labels.id == file_name].gender

    return author.values[0], gender.values[0]

In [205]:
def json_to_csv(path, labels_path, file_names):
    data = pd.DataFrame(columns = ['id', 'tweet', 'author', 'gender'])

    for file_name in file_names:
        # Verificar PATH
        file_name = check_path(file_name)
        
        # Leer archivo JSON basado en el nombre del archivo
        with open(f'{path}{file_name}.json') as f:
            json_file = json.load(f)
        
        # Obteniendo la cantidad de documentos que tiene el archivo JSON
        file_len = len(json_file['author']['documents']['document'])

        # Obtner las etiquetas del JSON
        author, gender = get_labels(labels_path, file_name)

        dict = {'id': [file_name] * file_len,
                'tweet': json_file['author']['documents']['document'],
                'author': [author] * file_len,
                'gender': [gender] * file_len}

        df = pd.DataFrame(data = dict)
        data = data.append(df)
        
    return data

In [208]:
final_data = json_to_csv(path_to_save_en, path_labels_en, file_names_en)

In [209]:
final_data

,id,tweet,author,gender
0,5ac1aafe060f39f2dc5ae45d14656630,RT @africanstudies: Book launch today for @Mag...,human,female
1,5ac1aafe060f39f2dc5ae45d14656630,RT @seam_project: Join the #MethodsMatter conv...,human,female
2,5ac1aafe060f39f2dc5ae45d14656630,These are the kind of University meetings that...,human,female
3,5ac1aafe060f39f2dc5ae45d14656630,In my office and on sale now (also with cheape...,human,female
4,5ac1aafe060f39f2dc5ae45d14656630,Such a pity I am not free tomorrow. But if you...,human,female
...,...,...,...,...
95,d7c88083b8790d7dfddd0cb5e84097f7,Merkel and Putin Sound Pragmatic Notes After Y...,bot,bot
96,d7c88083b8790d7dfddd0cb5e84097f7,Trump Slams Social-Media Companies for 'Censor...,bot,bot
97,d7c88083b8790d7dfddd0cb5e84097f7,Alexandria Ocasio-Cortez Criticized for Exclud...,bot,bot
98,d7c88083b8790d7dfddd0cb5e84097f7,Colorado family struggled with debt before hus...,bot,bot


In [210]:
final_data.to_csv('pan19-author-profiling-earlybirds-20190320.csv', 
                encoding='utf-8')